In [1]:
from pyspark import SparkContext, SparkConf
import findspark
import math
#Nome da aplicação. Identifica a aplicação no gerenciador do cluster
appName = 'Parte_A'

In [2]:
#Configuração do ambiente
findspark.init()
conf = SparkConf().setMaster('local').setAppName(appName)
conf.set('spark.executor.memory', '2G')
sc = SparkContext(conf=conf)
sc._jsc.hadoopConfiguration().set('textinputformat.record.delimiter', '\n\n')

In [9]:
#Criação de um RDD
def mapPairs(rdd):
    reg = json.loads(rdd)
    aux = []
    for key in reg.keys():
        if(key == "reviews"):
            for r in reg[key]:
                aux.append((reg['id'], ("review", r)))
        elif(key == "similar_items"):
            for s in reg[key]:
                aux.append((reg["id"], ("similar", s)))
        else:
            aux.append(((reg["id"], (key, reg[key]))))
    return aux

lines = sc.textFile("data-json").flatMap(mapPairs).cache()

In [4]:
#id do ASIN de um produto
id_asin = '0738700797'

'''
rdd = lines.flatMap(lambda line: line.split(" "))
words = rdd.map(lambda word: (word, 1))
counts = words.reduceByKey(lambda a, b: a + b)
counts.saveAsTextFile("counters")
'''

'\nrdd = lines.flatMap(lambda line: line.split(" "))\nwords = rdd.map(lambda word: (word, 1))\ncounts = words.reduceByKey(lambda a, b: a + b)\ncounts.saveAsTextFile("counters")\n'

In [ ]:
#questao b
pos_similares = []                                              
maior_venda = []

rdd_sales = lines.filter(lambda line: "salesrank: " in line)
sales_list = rdd_sales.collect()

asin = lines.filter(lambda item: 'ASIN: ' in item)
asin = asin.map(lambda item: item.split()[3])
asin = asin.collect()  
pos = asin.index(id_asin)-1 

similar = lines.filter(lambda item: 'similar: ' in item)
similar = similar.collect()    

prod_sim = similar[pos].split("similar: ")[1] 
prod_similares = prod_sim.split()                  


for i in range(len(prod_similares)-1):
    pos_similares.append(asin.index(prod_similares[i+1]) )    

for i in range(len(pos_similares)):
    if(int(sales_list[pos_similares[i]].split()[1]) < int(sales_list[pos].split()[1]) ):
        maior_venda.append("ASIN: "+asin[pos_similares[i+1]] + sales_list[ pos_similares[i] ] )

print(maior_venda)

In [10]:
id = input("Digite o id do produto a ser procurado: ")

Digite o id do produto a ser procurado: 2


In [ ]:
from pprint import pprint
similar_with_input = lines.filter(lambda x: x[0] == id and x[1][0] == "similar")

pprint(similar_with_input.collect())
